In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import decomposition as skdc
from sklearn import manifold as skmf
import matplotlib.colors as mpclrs
import sys
sys.path.insert(0, '/ru-auth/local/home/ezheng/temp')
import GTF
import seaborn as sns
from scipy import stats as spstats

In [2]:
safered = "#920000"
safegreen = "#009292"
safeviolet = "#490092"
safe_threecolor = [safeviolet, safegreen, safered]

In [3]:
# from https://stackoverflow.com/questions/22987015/slice-pandas-dataframe-by-multiindex-level-or-sublevel
# some syntactic sugar to deal with inane pandas

def filter_by(df, constraints):
    """Filter MultiIndex by sublevels."""
    indexer = [constraints[name] if name in constraints else slice(None)
               for name in df.index.names]
    return df.loc[tuple(indexer)] if len(df.shape) == 1 else df.loc[tuple(indexer),]

pd.Series.filter_by = filter_by
pd.DataFrame.filter_by = filter_by

In [4]:
minspeciesorder = ['dm', 'droSec',
       'droEre', 'droSuz',
       'droBip', 
       'droRho', 'droFic',
       'droMir', 'droWil',
       'droGri', 
       'musDom', 'anoGam', 'apiMel', 'triCas']

In [6]:
FB2_TPM = pd.read_pickle('../addlsourcedata/FB2_supermultiTPM.pandas.pic.gz')
modENCODE_TPM = pd.read_pickle('../addlsourcedata/modENCODE_supermultiTPM.pandas.pic.gz')
modENCODE_TPM.index.rename('Gene ID', level=0, inplace=True)

In [7]:
supermultiTPM = pd.merge(FB2_TPM, modENCODE_TPM, how='outer', left_index=True, right_index=True)

In [8]:
FB2_TPM.filter_by({'gene_class':'novel_candidate'}).shape

(1719, 105)

## sigTPM

In [10]:
bool_transform = {True: '1', False: '2'}
bool_dict = {'1': True, '2': False}

In [11]:
# generate a series of TPMs

sigTPM_thres = 0.1
sht_sigTPM = (supermultiTPM.filter_by({'gene_class':'novel_candidate'})
    #.groupby(level=['gene_symbol','seqname']).sum()
     .groupby(level=['gene_symbol']).sum()
    #.loc[lambda x: x.max(axis='columns') >= 1]
    .max(axis='columns'))
print(sht_sigTPM.shape)
LCA_sigTPM = (sht_sigTPM
                  .pipe(lambda s: s > sigTPM_thres)
                  .replace(bool_transform))
print(LCA_sigTPM.head())

# LCA_sigTPM:
#     1: False
#     2: True

(1056,)
gene_symbol
10.1      1
100.1     1
1001.1    1
1004.1    1
101.1     1
dtype: object


## monophyly

In [13]:
monophyletic = pd.read_pickle('../addlsourcedata/monophyletic.pandas.pic.gz')
sht_monophyletic = (monophyletic.reset_index()
                        .sort_values(['EBZid', 'midid'], ascending=True)
                        .drop_duplicates('EBZid', keep='first')
                        .set_index(['EBZid'])
                        .monophyletic)
print(sht_monophyletic.shape)
LCA_monophyletic = sht_monophyletic.replace(bool_transform)
# LCA_monophyletic:
#     1: False
#     2: True
LCA_monophyletic.head()

(1020,)


EBZid
10.1      2
100.1     2
1001.1    2
1004.1    1
101.1     2
Name: monophyletic, dtype: object

## age tier

In [14]:
furthest = pd.read_pickle('../addlsourcedata/furthest.pandas.pic.gz')

# squash surplus nodes
furthest = (furthest
            .replace('droSim', 'droSec')
            .replace('droBia', 'droSuz')
            .replace('droYak', 'droEre')
            .replace('droAna', 'droBip')
            .replace('droEug', 'droRho')
            .replace('droEle', 'droRho')
            .replace('droKik', 'droRho')
            .replace('droTak', 'droRho')
            .replace('droPse', 'droMir')
            .replace('droPer', 'droMir')
            .replace('droVir', 'droGri')
            .replace('droMoj', 'droGri')
            .replace('droAlb', 'droGri'))

speciesorder = ['dm', 'droSim', 'droSec', 'droYak',
       'droEre', 'droBia', 'droSuz',
       'droAna', 'droBip', 'droEug',
       'droEle', 'droKik', 'droTak',
       'droRho', 'droFic', 'droPse',
       'droPer', 'droMir', 'droWil',
       'droVir', 'droMoj', 'droAlb',
       'droGri', 
       'musDom', 'anoGam', 'apiMel', 'triCas']

minspeciesorder = ['dm', 'droSec',
       'droEre', 'droSuz',
       'droBip', 
       'droRho', 'droFic',
       'droMir', 'droWil',
       'droGri', 
       'musDom', 'anoGam', 'apiMel', 'triCas']

sht_furthest = furthest.reset_index().set_index('EBZid').furthest
print(sht_furthest.shape)
print(sht_furthest.head())

LCA_furthest_transform = {species: str(k+1) for k, species in enumerate(minspeciesorder)}
LCA_furthest_dict = {str(k+1): species for k, species in enumerate(minspeciesorder)}

LCA_furthest = sht_furthest.replace(LCA_furthest_transform)
print(LCA_furthest.shape)
LCA_furthest.head()

(1020,)
EBZid
10.1      droRho
100.1     droFic
1001.1    droMir
1004.1    droSuz
101.1     droFic
Name: furthest, dtype: object
(1020,)


EBZid
10.1      6
100.1     7
1001.1    8
1004.1    4
101.1     7
Name: furthest, dtype: object

In [15]:
single_blosum62_score_table = pd.read_pickle('~/temp/single_blosum62.pandas.pic.gz')
metadata = pd.read_csv('/ru-auth/local/home/ezheng/results/Dmel_MSannot/fin_finalproteins_classes.csv', index_col=0)

## conservation predictions

In [16]:
phyloP = (pd.read_csv('/ru-auth/local/home/ezheng/results/Dmel_MSannot/EBZ-orf-phyloP_byEBZ_ID.txt')
    .assign(EBZid=lambda x: x['Target'].str.extract(r'(?<=EBZ_)([0-9\.]+)(?=-|_)')))

phastcons = (pd.read_csv('/ru-auth/local/home/ezheng/results/Dmel_MSannot/EBZ-orf-phastcons_byEBZ_ID.txt')
    .assign(EBZid=lambda x: x['Target'].str.extract(r'(?<=EBZ_)([0-9\.]+)(?=-|_)')))

In [17]:
conservation = pd.merge(phyloP, phastcons, left_on=['EBZid', 'Target'], right_on=['EBZid', 'Target'])
conservation = conservation[conservation.EBZid.isin(metadata.EBZ_IDnew.astype(str))].set_index('EBZid')

In [18]:
LCA_phastCons_labels = ['noncons', 'amb', 'cons']
LCA_phastCons_transform = {'noncons':'1', 'amb':'2', 'cons':'3'}

sht_phastCons = conservation.phastCons
LCA_phastCons = pd.cut(conservation.phastCons, bins=[0, 0.2, 0.8, 1.0], labels=LCA_phastCons_labels)
print(LCA_phastCons.value_counts())
LCA_phastCons.shape
LCA_phastCons = LCA_phastCons.replace(LCA_phastCons_transform)
LCA_phastCons.value_counts()

amb        474
cons       364
noncons    136
Name: phastCons, dtype: int64


2    474
3    364
1    136
Name: phastCons, dtype: int64

In [19]:
LCA_phyloP_labels = ['noncons', 'amb', 'sigcons']
LCA_phyloP_transform = {'noncons':'1', 'amb':'2', 'sigcons':'3'}

sht_phyloP = conservation.phyloP
LCA_phyloP = pd.cut(conservation.phyloP, bins=[-5, 0, 1.30, 100], labels=LCA_phyloP_labels)
print(LCA_phyloP.value_counts())
LCA_phyloP.shape
LCA_phyloP = LCA_phyloP.replace(LCA_phyloP_transform)
LCA_phyloP.value_counts()

amb        485
sigcons    443
noncons     63
Name: phyloP, dtype: int64


2    485
3    443
1     63
Name: phyloP, dtype: int64

## location

In [20]:
location_properties = metadata.set_index('EBZ_IDnew')[['length', 'sense', 'antisense', 'intergenic']]

def classify_location(s):
    if (s.intergenic > 0) and (s.sense == 0) and (s.antisense == 0):
        return 'intergenic'
    elif (s.antisense > 0) and (s.sense == 0) and (s.intergenic == 0):
        return 'antisense'
    elif (s.sense > 0) and (s.antisense == 0) and (s.intergenic == 0):
        return 'sense'
    else:
        return 'mixed'

sht_location = location_properties.apply(classify_location, axis=1)

LCA_location_transform = {  'intergenic': '1',
                            'antisense':  '2',
                            'sense':      '3',
                            'mixed':      '4' }
LCA_location_dict = {   '1': 'intergenic',
                        '2': 'antisense',
                        '3': 'sense',
                        '4': 'mixed'   }

LCA_location = sht_location.replace(LCA_location_transform)
LCA_location.index = LCA_location.index.astype(str)
LCA_location.index.name = 'gene_symbol'
print(LCA_location.head())

sht_location.index = sht_location.index.astype(str)
sht_location.index.name = 'gene_symbol'
sht_location.head()

gene_symbol
2.1     1
4.1     2
9.1     1
10.1    2
20.1    2
dtype: object


gene_symbol
2.1     intergenic
4.1      antisense
9.1     intergenic
10.1     antisense
20.1     antisense
dtype: object

## length

In [21]:
LCA_length_dict = {'1':'[0, 20)',
                   '2':'[20, 50)',
                   '3':'50+'}
sht_length = metadata.set_index('EBZ_IDnew')['length']
sht_length.index.name = 'gene_symbol'
sht_length.index = sht_length.index.astype(str)
LCA_length = pd.cut(metadata.set_index('EBZ_IDnew')['length'], bins=[0, 20, 50, 999999], labels=['1','2','3'])
LCA_length.index.name = 'gene_symbol'
LCA_length.index = LCA_length.index.astype(str)
LCA_length.head()

gene_symbol
2.1     2
4.1     2
9.1     2
10.1    1
20.1    2
Name: length, dtype: category
Categories (3, object): [1 < 2 < 3]

## tau

In [22]:
def tau(input):
    #implementation of tau
  
    #define constants
    max_obs = max(input)
    num = len(input)
  
    #calculate it
    subtotal = (1 - input/max_obs)/(num-1)
    return (sum(subtotal))

In [23]:
adult_F_tissues =  ['FemaleAnal', 'FemaleBrain', 'FemaleCarcass',
                 'FemaleCrop', 'FemaleEye', 'FemaleHead', 'FemaleHindgut',
                 'FemaleMatedSpermatheca', 'FemaleMidgut', 'FemaleOvary',
                 'FemaleSalivary', 'FemaleTubule', 'FemaleVirginSpermatheca',]
adult_M_tissues = ['MaleAccessory',
                 'MaleAnal', 'MaleBrain', 'MaleCarcass', 'MaleCrop', 'MaleEye',
                 'MaleHead', 'MaleHindgut', 'MaleMidgut', 'MaleSalivary', 'MaleTestis',
                 'MaleTubule']
larval_tissues = ['LarvalCNS', 'LarvalCarcass',
                  'LarvalFatBody', 'LarvalHindgut', 'LarvalMidgut', 'LarvalSalivary',
                  'LarvalTrachea', 'LarvalTubule']

In [24]:
embryo_stages = ['embryos, 0-2 hr', 'embryos, 2-4 hr', 'embryos, 4-6 hr',
                 'embryos, 6-8 hr', 'embryos, 8-10 hr',
                 'embryos, 10-12 hr', 'embryos, 12-14 hr', 'embryos, 14-16 hr',
                 'embryos, 16-18 hr', 'embryos, 18-20 hr', 
                 'embryos, 20-22 hr', 'embryos, 22-24 hr']

larval_stages = ['L1 stage larvae', 'L2 stage larvae',
                 'L3 stage larvae, 12 hr post-molt',
                 'L3 stage larvae, dark blue gut PS(1-2)',
                 'L3 stage larvae, light blue gut PS(3-6)',
                 'L3 stage larvae, clear gut PS(7-9)']

pupal_stages =  ['WPP', 'pupae, 12 hr after WPP',
                 'pupae, 24 hrs after WPP', 'pupae, 2 days after WPP', 
                 'pupae, 3 days after WPP', 'pupae, 4 days after WPP']

dev_stages = embryo_stages + larval_stages + pupal_stages

In [25]:
tau_adult_F = (supermultiTPM.groupby(level=['gene_symbol', 'gene_class']).sum()
                .groupby(level='tissue', axis=1).mean())[adult_F_tissues].apply(tau, axis=1)

tau_adult_M = (supermultiTPM.groupby(level=['gene_symbol', 'gene_class']).sum()
                .groupby(level='tissue', axis=1).mean())[adult_M_tissues].apply(tau, axis=1)

In [26]:
tau_embryo = (supermultiTPM.groupby(level=['gene_symbol', 'gene_class']).sum()
                .groupby(level='tissue', axis=1).mean())[embryo_stages].apply(tau, axis=1)

tau_larval = (supermultiTPM.groupby(level=['gene_symbol', 'gene_class']).sum()
                .groupby(level='tissue', axis=1).mean())[larval_stages].apply(tau, axis=1)

tau_pupal  = (supermultiTPM.groupby(level=['gene_symbol', 'gene_class']).sum()
                .groupby(level='tissue', axis=1).mean())[pupal_stages].apply(tau, axis=1)

tau_dev    = (supermultiTPM.groupby(level=['gene_symbol', 'gene_class']).sum()
                .groupby(level='tissue', axis=1).mean())[dev_stages].apply(tau, axis=1)

In [27]:
tau_thres = 0.8

# tissue-specific (tau > 0.8) in EITHER M OR F
LCA_tau = (
    (tau_adult_M.filter_by({'gene_class':'novel_candidate'})
    .pipe(lambda s: s > tau_thres)) 
    | 
    (tau_adult_F.filter_by({'gene_class':'novel_candidate'})
    .pipe(lambda s: s > tau_thres))
)

print(LCA_tau.shape)
LCA_tau = LCA_tau.replace(bool_transform)
# LCA_tau:
#     1: False -- not tissue specific in both M and F
#     2: True -- tissue specific in either M or F
print(LCA_tau.head())
print(LCA_tau.value_counts())

sht_tau = (
    pd.concat([tau_adult_M.filter_by({'gene_class':'novel_candidate'}),
               tau_adult_F.filter_by({'gene_class':'novel_candidate'})], axis=1)
    .mean(axis=1)
)

print(sht_tau.shape)
print(sht_tau.head())

(1056,)
gene_symbol
10.1      1
100.1     1
1001.1    1
1004.1    1
101.1     1
dtype: object
1    762
2    294
dtype: int64
(1056,)
gene_symbol
10.1      1.000000
100.1     0.855625
1001.1    0.937958
1004.1    1.000000
101.1     0.949129
dtype: float64


In [28]:
LCA_dev_tau = (tau_dev.filter_by({'gene_class':'novel_candidate'})
    .pipe(lambda s: s > tau_thres))

print(LCA_dev_tau.shape)
LCA_dev_tau = LCA_dev_tau.replace(bool_transform)
# LCA_dev_tau:
#     1: False -- not tissue specific in both M and F
#     2: True -- tissue specific in either M or F
print(LCA_dev_tau.head())
print(LCA_dev_tau.value_counts())

sht_dev_tau = (tau_dev.filter_by({'gene_class':'novel_candidate'}))

print(sht_dev_tau.shape)
print(sht_dev_tau.head())

(1056,)
gene_symbol
10.1      1
100.1     2
1001.1    1
1004.1    2
101.1     1
dtype: object
1    668
2    388
dtype: int64
(1056,)
gene_symbol
10.1      0.942800
100.1     0.655067
1001.1    0.851240
1004.1         NaN
101.1     0.942865
dtype: float64


## table

In [29]:
LCA_location.name = 'genomic'
LCA_sigTPM.name = 'sigTPM'
LCA_tau.name = 'tau'
LCA_dev_tau.name = 'dev_tau'
LCA_phastCons.name = 'phastCons'
LCA_phyloP.name = 'phyloP'
LCA_length.name = 'length'

In [30]:
LCAtable_index = pd.DataFrame([LCA_furthest, LCA_location, LCA_phyloP, LCA_phastCons, LCA_length, LCA_monophyletic, LCA_sigTPM, LCA_tau, LCA_dev_tau]
                  ).T
LCAtable_index = LCAtable_index.loc[metadata.EBZ_IDnew.astype(str)]
LCAtable = LCAtable_index.fillna('NA').reset_index().drop('index',axis=1)
LCAtable.shape

(993, 9)

In [31]:
sht_location.name = 'genomic'
sht_sigTPM.name = 'sigTPM'
sht_tau.name = 'tau'
sht_dev_tau.name = 'dev_tau'
sht_phastCons.name = 'phastCons'
sht_phyloP.name = 'phyloP'
sht_length.name = 'length'

sht_table = pd.DataFrame([sht_furthest, sht_location, sht_phyloP, sht_phastCons, sht_length, sht_monophyletic, sht_sigTPM, sht_tau, sht_dev_tau]
                  ).T#.dropna()
sht_table = sht_table.loc[metadata.EBZ_IDnew.astype(str)]
sht_table.shape

(993, 9)

In [45]:
if False:
    LCAtable.to_csv('../addlsourcedata/LCA_factors_coded.csv')
    sht_table.to_csv('../addlsourcedata/LCA_factors_original.csv')
    LCAtable_index.to_pickle('../addlsourcedata/LCA_factors_indexed.pd.pic.gz')